<a href="https://colab.research.google.com/github/RutujKhare1/CS6910_Assignment3/blob/main/FDL_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import pandas as pd
import torch
import random
import torch.nn as nn

In [ ]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_train.csv', names=['eng','hin'])

In [ ]:
eng_words = df['eng'].copy()
eng_maxlen = len(max(df['eng'], key=len))
for i in range(len(eng_words)):
  l = len(eng_words[i])
  eng_words[i] = eng_words[i] + "*"*(eng_maxlen - l + 2)

In [ ]:
hin_words = df['hin'].copy()
hin_maxlen = len(max(df['hin'], key=len))
for i in range(len(hin_words)):
  l = len(hin_words[i])
  hin_words[i] = "#" + hin_words[i] + "*"*(hin_maxlen - l + 2)

In [ ]:
unique_eng_letters = set(''.join(eng_words))
unique_hin_letters = set(''.join(hin_words))
int_to_eng = dict(enumerate(unique_eng_letters))
eng_to_int = {char: ind for ind, char in int_to_eng.items()}

int_to_hin = dict(enumerate(unique_hin_letters))
hin_to_int = {char: ind for ind, char in int_to_hin.items()}

In [ ]:
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  index_eng_words.append(index_eng_word)

In [ ]:
index_hin_words = []
for hin_word in hin_words:
  index_hin_word = [hin_to_int[i] for i in hin_word]
  index_hin_words.append(index_hin_word)

In [ ]:
tensor_eng = torch.tensor(index_eng_words).to(device_name)
tensor_hin = torch.tensor(index_hin_words).to(device_name)
tensor_hin

tensor([[60, 18, 39,  ..., 23, 23, 23],
        [60, 13,  3,  ..., 23, 23, 23],
        [60, 56,  3,  ..., 23, 23, 23],
        ...,
        [60,  1, 39,  ..., 23, 23, 23],
        [60, 39, 12,  ..., 23, 23, 23],
        [60,  1, 17,  ..., 23, 23, 23]], device='cuda:0')

In [ ]:
class GRU_Encoder(nn.Module):
  def __init__(self, input_size, hid_size, num_of_enc_layers, emb_size, batch_size, dropout, bi_direct):
    super(GRU_Encoder, self).__init__()
    self.input_size = input_size
    self.hid_size = hid_size
    self.num_of_enc_layers = num_of_enc_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.dropout = dropout
    self.embedding = nn.Embedding(input_size, emb_size)
    # print("IS:{} ES:{}".format(input_size, emb_size))
    self.gru = nn.GRU(emb_size, hid_size, num_of_enc_layers, bidirectional = bi_direct, dropout = dropout)

  def forward(self, input_data, hidden):
    input_data = input_data.T
    # print(input_data.shape)
    embed = self.embedding(input_data).to(device_name)
    # print(embed.shape,hidden.shape)
    # embed = embed.view(-1, self.batch_size, self.hid_size)
    output, hidden = self.gru(embed, hidden)
    # if(self.bi_direct):
    #   print("bir\n")
    #   hidden = hidden.resize(2, self.num_of_enc_layers, self.batch_size, self.hid_size)
    #   print(hidden.shape)
    #   hidden = torch.add(hidden[0], hidden[1])/2
    #   print(hidden.shape)
    return output, hidden
  
  def initialiseHidden(self):
    if(self.bi_direct):
      return torch.zeros(2*self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
    else:
      return torch.zeros(self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
  

In [ ]:
class GRU_Decoder(nn.Module):
  def __init__(self, op_size, num_of_dec_layers, hid_size, batch_size, emb_size, dropout, bi_direct):
    super(GRU_Decoder, self).__init__()
    self.op_size = op_size
    self.hid_size = hid_size
    self.num_of_dec_layers = num_of_dec_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.embedding = nn.Embedding(op_size, emb_size)
    self.op = nn.Linear(2*hid_size, op_size) if (bi_direct) else nn.Linear(hid_size, op_size)
    self.softmax = nn.LogSoftmax(dim = 2)
    self.gru = nn.GRU(emb_size, hid_size, num_of_dec_layers, bidirectional = bi_direct, dropout = dropout)

  def forward(self, input_data, hidden):
    # print(input_data)
    embed = self.embedding(input_data)
    embed = embed.view(-1, self.batch_size, self.emb_size)
    out, hidden = self.gru(embed, hidden)
    # print(out.shape)
    temp = self.op(out)
    out = self.softmax(temp)
    return out, hidden

In [ ]:
def train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type):
  teacher_forcing = 0.5
  loss = 0
  for b in range(0, len(input_data), batch_size):
    x, y = input_data[b : b+batch_size], target_data[b : b+batch_size]
    temp = 0
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    # x = x.T
    # y = y.T
    t_step = len(x)
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
      
      if(num_of_dec_layers > num_of_enc_layers):
        num = num_of_dec_layers
        dec_hidden = enc_hidden
        while(num != num_of_enc_layers):
          dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
          num -= 1
      elif(num_of_dec_layers < num_of_enc_layers):
        dec_hidden = enc_hidden[-num_of_dec_layers:]
      else:
        dec_hidden = enc_hidden
      y = y.T
      dec_input = y[0]
      # print("Decoder IP : {}\nDecoder Hidden : {}".format(dec_input.shape, dec_hidden.shape))
      condition = False if random.random() > teacher_forcing else True
      if(condition):
        for i in range(1,len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          temp += loss_fn(torch.squeeze(dec_output), y[i])
          dec_input = y[i]
      else:
        for i in range(1,len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          prob, idx = dec_output.topk(1)
          temp += loss_fn(torch.squeeze(dec_output), y[i])
          dec_input = idx
    temp.backward()
    enc_optimizer.step()
    dec_optimizer.step()
    loss += temp
  # print(len(target_data))
  return loss.item() / len(target_data), encoder, decoder



In [ ]:
def eval(input_data, target_data, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type):
  out = []
  for b in range(0, len(input_data), batch_size):
    x, y = input_data[b : b+batch_size], target_data[b : b+batch_size]
    encoder.eval()
    decoder.eval()
    predicted_data = list()
    # x = x.T
    # y = y.T
    t_step = len(x)
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
      
      if(num_of_dec_layers > num_of_enc_layers):
        num = num_of_dec_layers
        dec_hidden = enc_hidden
        while(num != num_of_enc_layers):
          dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
          num -= 1
      elif(num_of_dec_layers < num_of_enc_layers):
        dec_hidden = enc_hidden[-num_of_dec_layers:]
      else:
        dec_hidden = enc_hidden
      y = y.T
      dec_input = y[0]

      for i in range(len(y)):
        dec_output, dec_hidden = decoder(dec_input, dec_hidden)
        prob, idx = dec_output.topk(1)
        idx = idx.squeeze()
        dec_input = idx
        predicted_data.append(idx.tolist())
      out.append(predicted_data)
  return out


In [ ]:
def training(input_data, input_size, target_data, target_size, max_input_size, epochs, batch_size, emb_size, num_of_enc_layers, num_of_dec_layers, hid_size, cell_type, bi_direct, enc_dropout, dec_dropout, beam_size):
  learning_rate = 0.001
  if(cell_type == "GRU"):
    encoder = GRU_Encoder(input_size, hid_size, num_of_enc_layers, emb_size, batch_size, enc_dropout, bi_direct).to(device_name)
    decoder = GRU_Decoder(target_size, num_of_dec_layers, hid_size, batch_size, emb_size, dec_dropout, bi_direct).to(device_name)
  
  enc_optimizer = torch.optim.Adam(encoder.parameters(), learning_rate)
  dec_optimizer = torch.optim.Adam(decoder.parameters(), learning_rate)
  loss_fn = nn.CrossEntropyLoss(reduction = 'sum')
  encoder.train()
  decoder.train()
  loss_list = []
  for i in range(epochs):
    loss, encoder, decoder = train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type)
    loss_list.append(loss/51200)
    print("Epoch : {} \tLoss : {}".format(i, loss))

  return encoder, decoder, num_of_enc_layers, num_of_dec_layers


In [ ]:
encoder, decoder, num_of_enc_layers, num_of_dec_layers = training(input_data = tensor_eng, 
                                                                  input_size = len(unique_eng_letters), 
                                                                  target_data = tensor_hin, 
                                                                  target_size = len(unique_hin_letters), 
                                                                  max_input_size = 24, 
                                                                  epochs = 20, 
                                                                  batch_size = 64, 
                                                                  emb_size = 32, 
                                                                  num_of_enc_layers = 3, 
                                                                  num_of_dec_layers = 1, 
                                                                  hid_size = 256, 
                                                                  cell_type = "GRU", 
                                                                  bi_direct = False, 
                                                                  enc_dropout = 0,
                                                                  dec_dropout = 0.2, 
                                                                  beam_size = 1)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch : 0 	Loss : 23.91609375
Epoch : 1 	Loss : 13.3956982421875
Epoch : 2 	Loss : 10.218072509765625
Epoch : 3 	Loss : 8.756336059570312
Epoch : 4 	Loss : 8.148068237304688
Epoch : 5 	Loss : 7.40233154296875
Epoch : 6 	Loss : 6.897642822265625
Epoch : 7 	Loss : 6.452665405273438
Epoch : 8 	Loss : 6.10385986328125
Epoch : 9 	Loss : 5.775126953125
Epoch : 10 	Loss : 5.308073120117188
Epoch : 11 	Loss : 5.147074584960937
Epoch : 12 	Loss : 5.013026733398437
Epoch : 13 	Loss : 4.829122924804688
Epoch : 14 	Loss : 4.34072021484375
Epoch : 15 	Loss : 4.1283642578125
Epoch : 16 	Loss : 4.005160217285156
Epoch : 17 	Loss : 3.856758728027344
Epoch : 18 	Loss : 3.7322817993164064
Epoch : 19 	Loss : 3.425146484375


In [ ]:
trained_pred = eval(tensor_eng, tensor_hin, encoder, decoder, num_of_enc_layers, num_of_dec_layers, 64, "GRU")

In [ ]:
temp = torch.tensor(trained_pred)
# temp.shape
temp = temp.view(23, 51200)
pred_up = temp.T.to(device_name)

In [ ]:
hin_maxlen

20

In [ ]:
pred_up

tensor([[18, 23, 23,  ..., 23, 23, 65],
        [13, 23, 23,  ..., 23, 23, 38],
        [56, 23, 23,  ..., 23, 23, 57],
        ...,
        [23, 23, 58,  ..., 23, 42, 23],
        [23, 23, 17,  ..., 62, 22, 23],
        [23,  3, 57,  ..., 65, 58, 23]], device='cuda:0')

In [ ]:
def calculateAccuracyUPD(y_pred, y_true):
  cnt = 0
  for i,j in zip(y_pred, y_true):
    cor = torch.eq(i, j)
    if(torch.mean(cor.float()).item() == 1.0):
      cnt += 1
  return cnt / len(y_pred)

In [ ]:
calculateAccuracyUPD(pred_up, tensor_hin)

0.0

In [ ]:
pred_up[0].shape

torch.Size([23])

0.0

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_test.csv', names=['eng','hin'])
eng_words = df_test['eng']
maxlen = len(max(df['eng'], key=len))
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  l = len(index_eng_word)
  index_eng_word.extend([0]*(maxlen-l+2))
  index_eng_words.append(index_eng_word)

tensor_eng_test = torch.tensor(index_eng_words).to(device_name)
tensor_hin_test = torch.tensor(index_hin_words).to(device_name)

In [ ]:
pred_value = eval(tensor_eng_test, tensor_hin_test, )

TypeError: ignored

# **Experimental Area**

In [ ]:
maxlen = len(max(df['eng'], key=len))

In [ ]:
len(tensor_eng)

In [ ]:
for i in df['eng']:
  if(len(i) > 24):
    print(i)

In [ ]:
a = [1,2,3,4]
a.append(0*4)
a

In [ ]:
eng_words

0        shastragaar***************
1        bindhya*******************
2        kirankant*****************
3        yagyopaveet***************
4        ratania*******************
                    ...            
51195    toned*********************
51196    mutanaazaa****************
51197    asahmaton*****************
51198    sulgaayin*****************
51199    anchuthengu***************
Name: eng, Length: 51200, dtype: object

In [ ]:
for i in pred_up:
  for j in i:
    print(int_to_hin[j.item()], end="")
  print()

श***र***ान***श***य्***द
ब***ल***बी***ा***ंल***ई
क**ी्***ज्**या****न***्
य***ी***ि्***त***िँ***्
र***ं****ो***ी****र***ी
व***स**ल*र**ली****म***ी
द**ाे****श***स***््***ा
स***क***ैा**लट***ीं**व्
म**ाप***लु*******िं****
स**गा***ों**ंव***डल***न
ब***ग****क**मन***रथ***्
त***े***न्**ुश****ब***ि
क**घ्****ज**ीु***िर***ड
इ***ौ***गा***क****इ***स
म**ेट***एब**चट***ीस***ग
अ**रम***़ु***ी***यन**व*
अ**कड***ा़***ग****ज***ा
ध**गद****्***न****र**ो*
अ***े***रै***य****र***ं
ए***े***ाे*******ाल***ढ
ब***म****ं*******दम***र
ब***ी***ेो***ह****प***ू
प***स***ील**रो***िस***ि
ह***े****श***य***यल***ई
ज***त****स***ी****ट***ं
प**कभ***टी***र***शक***ा
ज***ी***ोू**रय***यर***क
प***म***का***ट****र***न
आ*******वि***ा***ुक**़क
व**गन***तस***ल***लज***ा
च***्****े***ु***डश***र
म***र****े***व****्**्स
अ***ट****ं***क****्***ा
इ***ा***कव**यो****त***स
द**दू***टु***प***मि***ह
स***ै****स********ष***र
श**ठा**ीाक********स****
उ***त***वि********ि***े
अ***्****न***ा****व***े
अ***ग***ाॉ***ं****च***प
ख**वड***्ौ***व***ी़***ो
थ**प्***मा***म**

KeyboardInterrupt: ignored

In [ ]:
for x, y in zip(tensor_eng, tensor_hin):
  print(x,y)

In [ ]:
len(unique_hin_letters)

66

In [ ]:
a = [1,2],[3,4]
b = [1,1],[3,4]
ta = torch.tensor(a)
tb = torch.tensor(b)
calculateAccuracyUPD(a,b)